In [ ]:
import os
import lsst.utils
import lsst.daf.persistence as dafPersist
import pfs.drp.stella as drpStella

import lsst.afw.display as afwDisplay
display = afwDisplay.Display(1)

In [ ]:
dataDir = os.path.join(lsst.utils.getPackageDir("drp_stella_data"),'tests/data/PFS')
butler = dafPersist.Butler(dataDir)

In [ ]:
dataId = dict(visit=4, spectrograph=2, arm='r')
exposure = butler.get('postISRCCD', dataId)

In [ ]:
from pfs.drp.stella.utils import makeFiberTraceSet

dateObs = butler.queryMetadata("raw", ["dateObs"], dataId)[0]

fts = makeFiberTraceSet(butler.get("fiberTrace", dataId, dateObs=dateObs))

In [ ]:
maskPlane = "FIBERTRACE"
exposure.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "ORANGE")

ftMask = 1 << exposure.getMaskedImage().getMask().getMaskPlane(maskPlane)
for ft in fts.getTraces():
    drpStella.markFiberTraceInMask(ft, exposure.getMaskedImage().getMask(), ftMask)

display.mtv(exposure, title="parent")

In [ ]:
with display.Buffering():
    for ft in fts.getTraces():
        xCenters = ft.getXCenters()
        ftFunction = ft.getFiberTraceFunction()
        yLow = ftFunction.yCenter + ftFunction.yLow
        yHigh = ftFunction.yCenter + ftFunction.yHigh

        for y in range(yLow, yHigh):
            pointA = [xCenters[y-yLow]-0.5,y]
            pointB = [xCenters[y-yLow+1]-0.5,y+1]
            points = [pointA, pointB]
            display.line(points, ctype='red')